In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
# импортируем из модуля sklearn.model_selection 
# функцию train_test_split()
from sklearn.model_selection import train_test_split
# импортируем класс GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# импортируем функцию roc_auc_score()
from sklearn.metrics import roc_auc_score
# импортируем классы TimeSeriesSplit и GroupKFold,
# реализующие стратегии перекрестной проверки, и
# класс GridSearchCV для поиска гиперпараметров
from sklearn.model_selection import (
    TimeSeriesSplit, 
    GroupKFold,
    GridSearchCV
)

## Разбиение на обучающую и тестовую выборки, учитывающее временную структуру данных

In [2]:
# записываем CSV-файл в объект DataFrame
timeseries_data = pd.read_csv('Data/Flats.csv', 
                              encoding='UTF-8', 
                              sep=';', 
                              decimal=',')

In [3]:
# выводим первые 5 наблюдений датафрейма
timeseries_data.head()

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
0,1,Оловозаводская,12/1,6340,1,6,Кирпич,32.5,1450000,22.08.2016
1,1,Оловозаводская,12/1,6340,2,6,Кирпич,33.8,1650000,30.01.2017
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.2,2250000,28.10.2014
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.4,1960000,19.08.2014
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.3,1950000,28.02.2014


In [4]:
# преобразовываем в формат даты
timeseries_data['Date_Create'] = pd.to_datetime(timeseries_data['Date_Create'], 
                                                format='%d.%m.%Y')

In [5]:
# сортируем данные по дате сделки
# (от самой ранней к самой поздней)
timeseries_data = timeseries_data.sort_values(by='Date_Create', 
                                              ascending=1)
timeseries_data

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
11,2,Сухарная,76/3,3730,7,10,Кирпич,57.20,3300000,2005-07-06
5,1,Оловозаводская,12/1,6340,2,6,Кирпич,34.00,1700000,2011-11-23
12,3,Сухарная,76/3,3730,5,15,Кирпич,76.30,4950000,2013-12-06
16,3,Народная,16/1,3210,1,5,Панель,58.00,2900000,2014-01-30
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.30,1950000,2014-02-28
9,2,Сухарная,76/3,3730,14,15,Кирпич,57.10,3850000,2014-04-17
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.40,1960000,2014-08-19
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.20,2250000,2014-10-28
18,2,Чкалова,70/1,4390,4,9,Кирпич,47.00,3100000,2014-12-01
10,3,Сухарная,76/3,3730,13,15,Кирпич,74.20,5450000,2015-02-11


In [6]:
# выполняем разбиение на обучающую и тестовую выборки, 
# учитывающее временную структуру
X_time_tr, X_time_tst, y_time_tr, y_time_tst = train_test_split(timeseries_data.drop('Value_abs', axis=1), 
                                                                timeseries_data['Value_abs'], 
                                                                test_size=.3, 
                                                                shuffle=False)

In [7]:
# смотрим обучающую выборку
X_time_tr

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Date_Create
11,2,Сухарная,76/3,3730,7,10,Кирпич,57.2,2005-07-06
5,1,Оловозаводская,12/1,6340,2,6,Кирпич,34.0,2011-11-23
12,3,Сухарная,76/3,3730,5,15,Кирпич,76.3,2013-12-06
16,3,Народная,16/1,3210,1,5,Панель,58.0,2014-01-30
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.3,2014-02-28
9,2,Сухарная,76/3,3730,14,15,Кирпич,57.1,2014-04-17
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.4,2014-08-19
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.2,2014-10-28
18,2,Чкалова,70/1,4390,4,9,Кирпич,47.0,2014-12-01
10,3,Сухарная,76/3,3730,13,15,Кирпич,74.2,2015-02-11


In [8]:
# смотрим тестовую выборку
X_time_tst

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Date_Create
15,2,Народная,16/1,3210,1,5,Панель,44.40,2017-01-30
1,1,Оловозаводская,12/1,6340,2,6,Кирпич,33.80,2017-01-30
7,1,Шатурская,4,19820,17,17,Кирпич,35.27,2017-02-15
14,1,Сухарная,76/3,3730,13,15,Кирпич,37.10,2017-12-06
17,2,Чкалова,70/1,4390,1,9,Кирпич,52.00,2018-02-21
8,1,Планировочная,37,0,5,10,NaN,33.00,2018-08-30


In [9]:
# записываем CSV-файл в объект DataFrame
wf_data = pd.read_csv('Data/wellsfargo.csv', sep=';')
# выводим первые 5 наблюдений
wf_data.head()

,date,count_1,count_2,count_3,count_4,amount_1,amount_2,amount_3,count_5,count_6,count_7,amount_4,type,response
0,04.03.2017,1,1,0,0,10.0,10.0,0.0,1,0.0,1,10.0,O,0
1,21.10.2016,1,1,0,0,1000.0,1000.0,0.0,1,0.0,1,1000.0,O,0
2,28.12.2016,3,1,1,1,1000.0,1000.0,0.0,1,0.0,3,1000.0,O,0
3,08.01.2017,2,2,6,0,1000.0,1000.0,0.0,1,0.0,1,400.0,O,1
4,27.11.2017,1,1,0,0,1000.0,1000.0,0.0,1,0.0,1,1000.0,O,0


In [10]:
# преобразовываем в формат даты
wf_data['date'] = pd.to_datetime(wf_data['date'])

In [11]:
# сортируем данные по дате
# (от самой ранней к самой поздней)
wf_data = wf_data.sort_values(by='date', ascending=1)

In [12]:
# удаляем на месте date
wf_data.drop('date', inplace=True, axis=1)

In [13]:
# выполняем разбиение на обучающую и тестовую выборки, 
# учитывающее временную структуру
X_wf_train, X_wf_test, y_wf_train, y_wf_test = train_test_split(
    wf_data.drop('response', axis=1), wf_data['response'], 
    test_size=0.3, shuffle=False)

In [14]:
# выполняем дамми-кодирование
X_wf_train = pd.get_dummies(X_wf_train)
X_wf_test = pd.get_dummies(X_wf_test)

In [15]:
# создаем экземляр класса GradientBoostingClassifier
boost = GradientBoostingClassifier(random_state=152)
# обучаем модель
boost.fit(X_wf_train, y_wf_train)
# получаем вероятности положительного класса для обучающих данных
tr_proba = boost.predict_proba(X_wf_train)[:, 1]
# получаем вероятности положительного класса для тестовых данных
tst_proba = boost.predict_proba(X_wf_test)[:, 1]
# оцениваем качество модели на обучающих данных
print('AUC на обучающей выборке: {:.3f}'.format(
    roc_auc_score(y_wf_train, tr_proba)))
# оцениваем качество модели на тестовых данных
print('AUC на тестовой выборке: {:.3f}'.format(
    roc_auc_score(y_wf_test, tst_proba)))

AUC на обучающей выборке: 0.899
AUC на тестовой выборке: 0.844


## Перекрестная проверка, учитывающая временную структуру данных

### Перекрестная проверка расширяющимся окном

In [16]:
# создаем экземпляр класса TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
# создаем массив меток и массив признаков
y_ts = timeseries_data.pop('Value_abs').values
X_ts = timeseries_data.values

In [17]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций
for train_index, test_index in tscv.split(X_ts):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    print('Общее кол-во набл.: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3 4] TEST: [5 6 7 8 9]
Общее кол-во набл.: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10 11 12 13 14]
Общее кол-во набл.: 15
Обучающий набор: 10
Тестовый набор: 5

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [15 16 17 18 19]
Общее кол-во набл.: 20
Обучающий набор: 15
Тестовый набор: 5



### Перекрестная проверка расширяющимся окном (практический кейс)

In [18]:
# создаем массив меток и массив признаков
y_wf = wf_data.pop('response').values
X_wf = pd.get_dummies(wf_data).values

In [19]:
# создаем экземпляр класса TimeSeriesSplit,
# перекрестная проверка расширяющимся окном
time_cv = TimeSeriesSplit(n_splits=3)

In [20]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций, а также
# посмотрим AUC на разных временных срезах
for train_index, test_index in time_cv.split(X_wf):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_wf[train_index], X_wf[test_index]
    y_train, y_test = y_wf[train_index], y_wf[test_index]
    print('Общее кол-во набл.: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    boost.fit(X_train, y_train)
    train_proba = boost.predict_proba(X_train)[:, 1]
    test_proba = boost.predict_proba(X_test)[:, 1]
    print('AUC на обучающей выборке: {:.3f}'.format(
        roc_auc_score(y_train, train_proba)))
    print('AUC на тестовой выборке: {:.3f}'.format(
        roc_auc_score(y_test, test_proba)))
    print('')

TRAIN: [   0    1    2 ... 1247 1248 1249] TEST: [1250 1251 1252 ... 2497 2498 2499]
Общее кол-во набл.: 2500
Обучающий набор: 1250
Тестовый набор: 1250
AUC на обучающей выборке: 0.935
AUC на тестовой выборке: 0.834

TRAIN: [   0    1    2 ... 2497 2498 2499] TEST: [2500 2501 2502 ... 3747 3748 3749]
Общее кол-во набл.: 3750
Обучающий набор: 2500
Тестовый набор: 1250
AUC на обучающей выборке: 0.904
AUC на тестовой выборке: 0.854

TRAIN: [   0    1    2 ... 3747 3748 3749] TEST: [3750 3751 3752 ... 4997 4998 4999]
Общее кол-во набл.: 5000
Обучающий набор: 3750
Тестовый набор: 1250
AUC на обучающей выборке: 0.898
AUC на тестовой выборке: 0.844



### Перекрестная проверка скользящим окном

In [21]:
# фиксируем размер обучающего набора
tscv2 = TimeSeriesSplit(n_splits=3, max_train_size=5)

In [22]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций
for train_index, test_index in tscv2.split(X_ts):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3 4] TEST: [5 6 7 8 9]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [5 6 7 8 9] TEST: [10 11 12 13 14]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [10 11 12 13 14] TEST: [15 16 17 18 19]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5



### Перекрестная проверка скользящим окном (практический кейс)

In [23]:
# создаем экземпляр класса TimeSeriesSplit,
# перекрестная проверка скользящим окном
time_cv2 = TimeSeriesSplit(n_splits=3, max_train_size=1250)

In [24]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций, а также
# посмотрим AUC на разных временных срезах
for train_index, test_index in time_cv2.split(X_wf):
    print('TRAIN:', train_index[0], train_index[-1], 'TEST:', test_index[0], test_index[-1])
    X_train, X_test = X_wf[train_index], X_wf[test_index]
    y_train, y_test = y_wf[train_index], y_wf[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    boost.fit(X_train, y_train)
    train_proba = boost.predict_proba(X_train)[:, 1]
    test_proba = boost.predict_proba(X_test)[:, 1]
    print('AUC на обучающей выборке: {:.3f}'.format(
        roc_auc_score(y_train, train_proba)))
    print('AUC на тестовой выборке: {:.3f}'.format(
        roc_auc_score(y_test, test_proba)))
    print('')

TRAIN: 0 1249 TEST: 1250 2499
Общее кол-во наблюдений: 2500
Обучающий набор: 1250
Тестовый набор: 1250
AUC на обучающей выборке: 0.935
AUC на тестовой выборке: 0.834

TRAIN: 1250 2499 TEST: 2500 3749
Общее кол-во наблюдений: 2500
Обучающий набор: 1250
Тестовый набор: 1250
AUC на обучающей выборке: 0.947
AUC на тестовой выборке: 0.853

TRAIN: 2500 3749 TEST: 3750 4999
Общее кол-во наблюдений: 2500
Обучающий набор: 1250
Тестовый набор: 1250
AUC на обучающей выборке: 0.948
AUC на тестовой выборке: 0.830



###  Перекрестная проверка скользящим окном с блокировкой

In [25]:
# пишем класс BlockingTimeSeriesSplit
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [26]:
# применяем наш класс BlockingTimeSeriesSplit
btscv = BlockingTimeSeriesSplit(n_splits=3)
for train_index, test_index in btscv.split(X_ts):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3] TEST: [4 5]
Общее кол-во наблюдений: 6
Обучающий набор: 4
Тестовый набор: 2

TRAIN: [6 7 8 9] TEST: [10 11]
Общее кол-во наблюдений: 6
Обучающий набор: 4
Тестовый набор: 2

TRAIN: [12 13 14 15] TEST: [16 17]
Общее кол-во наблюдений: 6
Обучающий набор: 4
Тестовый набор: 2



### Проверка Walk Forward

In [27]:
# сформируем обучающий и тестовый набор
train_size = int(len(X_wf) * 0.66)
train, test = X_wf[0:train_size], X_wf[train_size:len(X_wf)]
# задаем минимальное количество наблюдений, с которого
# начнется проверка
n_train = 500
# задаем количество моделей (равно количеству наблюдений минус 
# минимальное количество наблюдений для старта проверки)
n_records = len(X_wf)
# выполняем проверку
for i in range(n_train, n_records):
    train, test = X_wf[0:i], X_wf[i:i+1]
    print('train=%d, test=%d' % (len(train), len(test)))

train=500, test=1
train=501, test=1
train=502, test=1
train=503, test=1
train=504, test=1
train=505, test=1
train=506, test=1
train=507, test=1
train=508, test=1
train=509, test=1
train=510, test=1
train=511, test=1
train=512, test=1
train=513, test=1
train=514, test=1
train=515, test=1
train=516, test=1
train=517, test=1
train=518, test=1
train=519, test=1
train=520, test=1
train=521, test=1
train=522, test=1
train=523, test=1
train=524, test=1
train=525, test=1
train=526, test=1
train=527, test=1
train=528, test=1
train=529, test=1
train=530, test=1
train=531, test=1
train=532, test=1
train=533, test=1
train=534, test=1
train=535, test=1
train=536, test=1
train=537, test=1
train=538, test=1
train=539, test=1
train=540, test=1
train=541, test=1
train=542, test=1
train=543, test=1
train=544, test=1
train=545, test=1
train=546, test=1
train=547, test=1
train=548, test=1
train=549, test=1
train=550, test=1
train=551, test=1
train=552, test=1
train=553, test=1
train=554, test=1
train=555,

train=2187, test=1
train=2188, test=1
train=2189, test=1
train=2190, test=1
train=2191, test=1
train=2192, test=1
train=2193, test=1
train=2194, test=1
train=2195, test=1
train=2196, test=1
train=2197, test=1
train=2198, test=1
train=2199, test=1
train=2200, test=1
train=2201, test=1
train=2202, test=1
train=2203, test=1
train=2204, test=1
train=2205, test=1
train=2206, test=1
train=2207, test=1
train=2208, test=1
train=2209, test=1
train=2210, test=1
train=2211, test=1
train=2212, test=1
train=2213, test=1
train=2214, test=1
train=2215, test=1
train=2216, test=1
train=2217, test=1
train=2218, test=1
train=2219, test=1
train=2220, test=1
train=2221, test=1
train=2222, test=1
train=2223, test=1
train=2224, test=1
train=2225, test=1
train=2226, test=1
train=2227, test=1
train=2228, test=1
train=2229, test=1
train=2230, test=1
train=2231, test=1
train=2232, test=1
train=2233, test=1
train=2234, test=1
train=2235, test=1
train=2236, test=1
train=2237, test=1
train=2238, test=1
train=2239, 

train=3686, test=1
train=3687, test=1
train=3688, test=1
train=3689, test=1
train=3690, test=1
train=3691, test=1
train=3692, test=1
train=3693, test=1
train=3694, test=1
train=3695, test=1
train=3696, test=1
train=3697, test=1
train=3698, test=1
train=3699, test=1
train=3700, test=1
train=3701, test=1
train=3702, test=1
train=3703, test=1
train=3704, test=1
train=3705, test=1
train=3706, test=1
train=3707, test=1
train=3708, test=1
train=3709, test=1
train=3710, test=1
train=3711, test=1
train=3712, test=1
train=3713, test=1
train=3714, test=1
train=3715, test=1
train=3716, test=1
train=3717, test=1
train=3718, test=1
train=3719, test=1
train=3720, test=1
train=3721, test=1
train=3722, test=1
train=3723, test=1
train=3724, test=1
train=3725, test=1
train=3726, test=1
train=3727, test=1
train=3728, test=1
train=3729, test=1
train=3730, test=1
train=3731, test=1
train=3732, test=1
train=3733, test=1
train=3734, test=1
train=3735, test=1
train=3736, test=1
train=3737, test=1
train=3738, 

### Перекрестная проверка расширяющимся окном с гэпом

In [28]:
# фиксируем размер гэпа
tscv3 = TimeSeriesSplit(n_splits=3, gap=2)
for train_index, test_index in tscv3.split(X_ts):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2] TEST: [5 6 7 8 9]
Общее кол-во наблюдений: 8
Обучающий набор: 3
Тестовый набор: 5

TRAIN: [0 1 2 3 4 5 6 7] TEST: [10 11 12 13 14]
Общее кол-во наблюдений: 13
Обучающий набор: 8
Тестовый набор: 5

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12] TEST: [15 16 17 18 19]
Общее кол-во наблюдений: 18
Обучающий набор: 13
Тестовый набор: 5



### Перекрестная проверка скользящим окном с гэпом

In [29]:
# фиксируем размер обучающего набора и размер гэпа
tscv4 = TimeSeriesSplit(n_splits=3, max_train_size=5, gap=1)
for train_index, test_index in tscv4.split(X_ts):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3] TEST: [5 6 7 8 9]
Общее кол-во наблюдений: 9
Обучающий набор: 4
Тестовый набор: 5

TRAIN: [4 5 6 7 8] TEST: [10 11 12 13 14]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [ 9 10 11 12 13] TEST: [15 16 17 18 19]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5



## Стратегия валидации для сочетания формата данных «один клиент – несколько наблюдений» и временного ряда

In [30]:
# записываем CSV-файл в объект DataFrame
toy_data = pd.read_csv('Data/toy_example.csv', sep=';')
toy_data

,Client,Date,Transaction,Status
0,Petrov,01.01.2011,12000,0
1,Petrov,01.01.2013,24000,0
2,Petrov,01.01.2012,34000,0
3,Ivanov,01.01.2013,10000,1
4,Ivanov,01.01.2012,45000,1
5,Ivanov,01.01.2011,15000,1
6,Sidorov,01.01.2012,18000,1
7,Sidorov,01.01.2011,69000,1
8,Sidorov,01.01.2013,80000,1


In [31]:
# преобразовываем в формат даты
toy_data['Date'] = pd.to_datetime(toy_data['Date'])
# сортируем по дате и перезадаем индекс без вставки
# индекса в качестве отдельного столбца
toy_data = toy_data.sort_values(by=['Date']).reset_index(drop=True)
toy_data

,Client,Date,Transaction,Status
0,Petrov,2011-01-01,12000,0
1,Ivanov,2011-01-01,15000,1
2,Sidorov,2011-01-01,69000,1
3,Petrov,2012-01-01,34000,0
4,Ivanov,2012-01-01,45000,1
5,Sidorov,2012-01-01,18000,1
6,Petrov,2013-01-01,24000,0
7,Ivanov,2013-01-01,10000,1
8,Sidorov,2013-01-01,80000,1


In [32]:
# пишем класс GroupKFoldPlusTimeSeriesSplit, выполняющий 
# проверку GroupKFold + TimeSeriesSplit
class GroupKFoldPlusTimeSeriesSplit():
    def __init__(self, n_splits, n_splits_time):
        self.n_splits = n_splits
        self.n_splits_time = n_splits_time
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        group_kfold = GroupKFold(n_splits=self.n_splits)
        for train_index, test_index in group_kfold.split(X, y, groups=groups):
            tscv = TimeSeriesSplit(n_splits=self.n_splits_time)
            for train_index_t, test_index_t in tscv.split(X, y):
                yield np.intersect1d(train_index, train_index_t), np.intersect1d(test_index, test_index_t)

In [33]:
# создаем экземпляр класса GroupKFoldPlusTimeSeriesSplit
custom_kfold = GroupKFoldPlusTimeSeriesSplit(n_splits=3, 
                                             n_splits_time=2)
# создаем массив признаков и массив меток
X_toy = toy_data.drop(columns=['Status'])
y_toy = toy_data['Status']
# задаем идентификатор меток
groups = toy_data['Client']

# взглянем на индексы наблюдений, попавших 
# в обучающий и тестовый наборы
for train_index, test_index in custom_kfold.split(
    X_toy, y_toy, groups=groups):
    print('---------')
    print("TRAIN:", train_index)
    print("TEST:", test_index)
    X_train, X_test = X_toy.iloc[train_index], X_toy.iloc[test_index]
    print('---')
    print('X_train\n', X_train)
    print('X_test\n', X_test)

---------
TRAIN: [0 1]
TEST: [5]
---
X_train
    Client       Date  Transaction
0  Petrov 2011-01-01        12000
1  Ivanov 2011-01-01        15000
X_test
     Client       Date  Transaction
5  Sidorov 2012-01-01        18000
---------
TRAIN: [0 1 3 4]
TEST: [8]
---
X_train
    Client       Date  Transaction
0  Petrov 2011-01-01        12000
1  Ivanov 2011-01-01        15000
3  Petrov 2012-01-01        34000
4  Ivanov 2012-01-01        45000
X_test
     Client       Date  Transaction
8  Sidorov 2013-01-01        80000
---------
TRAIN: [1 2]
TEST: [3]
---
X_train
     Client       Date  Transaction
1   Ivanov 2011-01-01        15000
2  Sidorov 2011-01-01        69000
X_test
    Client       Date  Transaction
3  Petrov 2012-01-01        34000
---------
TRAIN: [1 2 4 5]
TEST: [6]
---
X_train
     Client       Date  Transaction
1   Ivanov 2011-01-01        15000
2  Sidorov 2011-01-01        69000
4   Ivanov 2012-01-01        45000
5  Sidorov 2012-01-01        18000
X_test
    Client       

## Комбинированная проверка для настройки гиперпараметров, учитывающая временную структуру данных

In [34]:
# создаем экземпляр класса TimeSeriesSplit,
# перекрестная проверка скользящим окном
timecv = TimeSeriesSplit(n_splits=3, max_train_size=875)

In [35]:
# необходимо настроить проверку скользящим окном (параметр max_train_size) так,
# чтобы размеры обучающей и проверочной выборки были примерно
# одинаковыми и наблюдения в разных обучающих/проверочных выборках
# не перекрывались, для этого взглянем на индексы наблюдений, 
# попавших в обучающую и проверочную выборки, по каждой из 3 итераций
# поскольку работаем с датафреймами напрямую, для получения
# обучающей и проверочной выборок используем свойство .iloc
for train_index, test_index in timecv.split(X_wf_train):
    print('TRAIN:', train_index[0], train_index[-1], 'TEST:', test_index[0], test_index[-1])
    X_train, X_test = X_wf_train.iloc[train_index], X_wf_train.iloc[test_index]
    y_train, y_test = y_wf_train.iloc[train_index], y_wf_train.iloc[test_index]  
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающая выборка: %d' % (len(X_train)))
    print('Проверочная выборка: %d' % (len(X_test)))

TRAIN: 0 874 TEST: 875 1749
Общее кол-во наблюдений: 1750
Обучающая выборка: 875
Проверочная выборка: 875
TRAIN: 875 1749 TEST: 1750 2624
Общее кол-во наблюдений: 1750
Обучающая выборка: 875
Проверочная выборка: 875
TRAIN: 1750 2624 TEST: 2625 3499
Общее кол-во наблюдений: 1750
Обучающая выборка: 875
Проверочная выборка: 875


In [36]:
# создаем экземпляр класса GradientBoostingClassifier
boost_grid = GradientBoostingClassifier(random_state=42)

# задаем сетку гиперпараметров
param_grid = {'max_depth': [2, 4, 6, 8],
              'subsample': [0.6, 0.7, 0.8, 0.9, 1]}

# создаем экземпляр класса GridSearchCV
grid_search = GridSearchCV(boost_grid, 
                           param_grid, 
                           scoring='roc_auc', 
                           return_train_score=True,
                           n_jobs=-1, 
                           cv=timecv)

# запускаем решетчатый поиск
grid_search.fit(X_wf_train, y_wf_train)
# проверяем качество модели с комбинацией значений гиперпараметров, 
# давшей максимальное усредненное значение AUC, на тестовой выборке
test_score = roc_auc_score(y_wf_test, grid_search.predict_proba(X_wf_test)[:, 1])
# смотрим результаты решетчатого поиска
print('Наилучшие значения гиперпараметров: {}'.format(grid_search.best_params_))
print('Лучшее усредненное значение AUC cv: {:.2f}'.format(grid_search.best_score_))
print('AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: {:.2f}'.format(test_score))

Наилучшие значения гиперпараметров: {'max_depth': 2, 'subsample': 0.6}
Лучшее усредненное значение AUC cv: 0.84
AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: 0.85
